In [1]:
out = []
with open('../conll_dataset/aida-yago2-dataset/AIDA-YAGO2-dataset.tsv') as f:
    index = 1
    me = []
    ss = []
    first = True
    for line in f:
        if line.startswith('-DOCSTART-'):
            if first:
                first = False
                continue
            out.append([index, ' '.join(ss), list(set(me))])
            index += 1
            me = []
            ss = []
        else:
            line_spl = line.replace('\n', '').split('\t')
            ss.append(line_spl[0])
            if len(line_spl) > 4:
                if line_spl[1] == 'B':
                    me.append((line_spl[2], line_spl[4].replace('http://en.wikipedia.org/wiki/','')))
data = out
data[0]

[1,
 'EU rejects German call to boycott British lamb .  Peter Blackburn  BRUSSELS 1996-08-22  The European Commission said on Thursday it disagreed with German advice to consumers to shun British lamb until scientists determine whether mad cow disease can be transmitted to sheep .  Germany \'s representative to the European Union \'s veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer .  " We do n\'t support any such recommendation because we do n\'t see any grounds for it , " the Commission \'s chief spokesman Nikolaus van der Pas told a news briefing .  He said further scientific study was required and if it was found that action was needed it should be taken by the European Union .  He said a proposal last month by EU Farm Commissioner Franz Fischler to ban sheep brains , spleens and spinal cords from the human and animal food chains was a highly specific and precautionary mo

In [ ]:
for d in tqdm_notebook(data):
    # sentence of the target document
    sentence = d[1]
    
    # ts are target mentions on the document
    ts = [str(t[0]) for t in d[2]]
    true_entities = [str(t[1]).replace('_', ' ') for t in d[2]]
    
    # tokenize sentences by using target mentions
    # and model_probs is the output of get_probs function from the notebook you added
    tokenize = partial(en_tokenize, ts=ts)
    sent_splits, model_probs = solve_model_probs(sentence, tagger, tokenize=tokenize)
    
    # predicted entities that have the highest score of each mentions
    pred_entities = run(ts, sent_splits, model_probs, indices2title, type_oracle, trie, trie_index2indices_values, trie_index2indices_counts)
    
    # append result: true -> true entity, pred -> predicted entity
    results += [{'doc_id':d[0], 'mention':x, 'true': y, 'pred': z} for x,y,z in zip(ts, true_entities, pred_entities)]
    
df = pd.DataFrame(results)
matched = df['pred'] == df['true']
length = df['pred'].shape[0]
assert len(df['pred']) == len(df['true'])
accuracy = float(sum(matched))/float(length)